In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [172]:
import pandas as pd

df_temp=pd.read_csv('gdrive/MyDrive/Data/GlobalLandTemperaturesByCountry.csv') #Temperature df
df_co2=pd.read_csv('gdrive/MyDrive/Data/CO2emissionbycountries.csv',encoding= 'unicode_escape') #CO2 df

#df_t=pd.read_csv('gdrive/MyDrive/Data/city_temperature.csv') 

In [173]:
df_co2

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km²
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km²
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km²
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km²
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km²
...,...,...,...,...,...,...,...,...,...
59615,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²
59616,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²
59617,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²
59618,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²


In [174]:
from datetime import datetime
df_temp['dt']= pd.to_datetime(df_temp['dt'])

df_temp['Year'] = pd.DatetimeIndex(df_temp['dt']).year
df_temp = df_temp.drop(columns=['dt','AverageTemperatureUncertainty'])
df_temp = df_temp[['Year', 'Country', 'AverageTemperature']]

df_temp = df_temp[df_temp['Year'] >= 1950]
df_temp = df_temp.reset_index()
print(df_temp)



         index  Year   Country  AverageTemperature
0         2474  1950     Åland              -5.611
1         2475  1950     Åland              -1.804
2         2476  1950     Åland               0.090
3         2477  1950     Åland               3.630
4         2478  1950     Åland               8.106
...        ...   ...       ...                 ...
185889  577457  2013  Zimbabwe              19.059
185890  577458  2013  Zimbabwe              17.613
185891  577459  2013  Zimbabwe              17.000
185892  577460  2013  Zimbabwe              19.759
185893  577461  2013  Zimbabwe                 NaN

[185894 rows x 4 columns]


In [175]:
df_temp2=df_temp.fillna(method='ffill')
df_temp2

,index,Year,Country,AverageTemperature
0,2474,1950,Åland,-5.611
1,2475,1950,Åland,-1.804
2,2476,1950,Åland,0.090
3,2477,1950,Åland,3.630
4,2478,1950,Åland,8.106
...,...,...,...,...
185889,577457,2013,Zimbabwe,19.059
185890,577458,2013,Zimbabwe,17.613
185891,577459,2013,Zimbabwe,17.000
185892,577460,2013,Zimbabwe,19.759


In [176]:
result = df_temp2.groupby(['Country','Year'])['AverageTemperature'].mean()

# Display result
type(result)
df_temp3 = result
df_temp3.to_frame().reset_index()
#df_temp3['AverageTemperature'].round(2)
df_temp3

Country      Year
Afghanistan  1950    13.043500
             1951    13.967750
             1952    14.175417
             1953    14.650750
             1954    13.691333
                       ...    
Åland        2009     6.489083
             2010     4.861917
             2011     7.170750
             2012     6.063917
             2013     7.362556
Name: AverageTemperature, Length: 15552, dtype: float64

In [ ]:
df_co2.head(5)

In [177]:
#Dropping data with CO2 emission equal 0
#df= df_co2[df_co2['CO2 emission (Tons)'] != 0]
#11880 
df = df_co2
df = df[df['Year'] >= 1959]

df = df[df['Year'] < 2014]

df.fillna(method='ffill')

df=df.drop(columns=['Code','Calling Code','Area','Population(2022)','% of World','Density(km2)'])
df = df.rename(columns={'CO2 emission (Tons)': 'CO2'})
df

,Country,Year,CO2
209,Afghanistan,1959,1839179.0
210,Afghanistan,1960,2253064.0
211,Afghanistan,1961,2743862.0
212,Afghanistan,1962,3432456.0
213,Afghanistan,1963,4139192.0
...,...,...,...
59608,Zimbabwe,2009,664256525.0
59609,Zimbabwe,2010,672134841.0
59610,Zimbabwe,2011,681878824.0
59611,Zimbabwe,2012,689761713.0


In [178]:
#Left outer join exclusion on country with data surveys starting from 1959
df_final=pd.merge(df,df_temp3,on=['Country','Year'],how="inner",indicator=True)
#df_final=df_final[df_final['_merge']=='left_only']
# resetting the DataFrame index
df_final = df_final.reset_index()

df_final=df_final.drop(columns=['index','_merge'])
df_final = df_final.round(2)
df_final['CO2'] = df_final['CO2'].astype(int)
df_final

,Country,Year,CO2,AverageTemperature
0,Afghanistan,1959,1839179,14.15
1,Afghanistan,1960,2253064,13.99
2,Afghanistan,1961,2743862,14.06
3,Afghanistan,1962,3432456,13.77
4,Afghanistan,1963,4139192,15.03
...,...,...,...,...
10225,Zimbabwe,2009,664256525,21.38
10226,Zimbabwe,2010,672134841,21.99
10227,Zimbabwe,2011,681878824,21.60
10228,Zimbabwe,2012,689761713,21.52


In [170]:
#Preparing data by dropping duplicates to visualize in which years surveys start.

df= df_co2[df_co2['CO2 emission (Tons)'] != 0]
df_single=df
df_single=df_single.drop_duplicates(subset=['Country'])
df_single

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
199,Afghanistan,AF,93,1949,14656.0,41128771.0,652230.0,0.40%,63/km²
454,Albania,AL,355,1933,7328.0,2842321.0,28748.0,0.00%,99/km²
708,Algeria,DZ,213,1916,3664.0,44903225.0,2381741.0,1.60%,19/km²
1053,Andorra,AD,376,1990,406704.0,79824.0,468.0,0.00%,171/km²
1284,Angola,AO,244,1950,186864.0,35588987.0,1246700.0,0.80%,29/km²
...,...,...,...,...,...,...,...,...,...
58407,Vietnam,VN,84,1892,212512.0,98186856.0,331212.0,0.20%,296/km²
58787,Wallis and Futuna,WF,681,2001,14656.0,NaN,NaN,NaN,NaN
59007,Yemen,YE,967,1950,58624.0,33696614.0,527968.0,0.40%,64/km²
59278,Zambia,ZM,260,1950,2330187.0,20017675.0,752612.0,0.50%,27/km²


In [10]:
#Filtering country with data surveys starting from 1959
df_single = df_single[df_single['Year'] >= 1959]
print(df_single)

                        Country Code Calling Code  Year  CO2 emission (Tons)  \
1053                    Andorra   AD          376  1990             406704.0   
1595                   Anguilla   AI        1-264  1990              51296.0   
1863                 Antarctica   AQ          672  1987               3664.0   
6453                     Bhutan   BT          975  1970               3664.0   
7268                   Botswana   BW          267  1972              21984.0   
11060          Christmas Island   CX           61  1970              69616.0   
12143              Cook Islands   CK          682  1969              10992.0   
27582                  Kiribati   KI          686  1961               7328.0   
27900                    Kosovo   XK          383  2008            7386624.0   
29508                   Lesotho   LS          266  1990            1472928.0   
30321             Liechtenstein   LI          423  1990             198968.0   
32199                  Maldives   MV    

In [11]:
#Left outer join exclusion on country with data surveys starting from 1959
df_final=pd.merge(df,df_single[['Country']],on=['Country'],how="outer",indicator=True)
df_final=df_final[df_final['_merge']=='left_only']
# resetting the DataFrame index
df_final = df_final.reset_index()

In [12]:
df_new=df_final[df_final['Year'] >= 1959]

df_new = df_new.reset_index()

In [13]:
df_new

,level_0,index,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2),_merge
0,10,10,Afghanistan,AF,93,1959,1839179.0,41128771.0,652230.0,0.40%,63/km²,left_only
1,11,11,Afghanistan,AF,93,1960,2253064.0,41128771.0,652230.0,0.40%,63/km²,left_only
2,12,12,Afghanistan,AF,93,1961,2743862.0,41128771.0,652230.0,0.40%,63/km²,left_only
3,13,13,Afghanistan,AF,93,1962,3432456.0,41128771.0,652230.0,0.40%,63/km²,left_only
4,14,14,Afghanistan,AF,93,1963,4139192.0,41128771.0,652230.0,0.40%,63/km²,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
11837,20533,21702,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²,left_only
11838,20534,21703,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²,left_only
11839,20535,21704,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²,left_only
11840,20536,21705,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²,left_only


In [ ]:
df_final.info(verbose=True)

In [14]:
df_new=df_new.drop(columns=['index','level_0','Calling Code','_merge','Area','Population(2022)','% of World'])

In [58]:
df_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11842 entries, 0 to 11841
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              11842 non-null  object 
 1   Code                 11470 non-null  object 
 2   Year                 11842 non-null  int64  
 3   CO2 emission (Tons)  11842 non-null  float64
 4   Density(km2)         10974 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 462.7+ KB


In [ ]:

check_nan =df_new['Density(km2)'].isnull()
print(check_nan)
df_new.loc[df_new['Density(km2)'].isnull()]

# printing the result

0        False
1        False
2        False
3        False
4        False
         ...  
11837    False
11838    False
11839    False
11840    False
11841    False
Name: Density(km2), Length: 11842, dtype: bool


,Country,Code,Year,CO2 emission (Tons),Density(km2)
434,Aruba,AW,1959,10044560.0,NaN
435,Aruba,AW,1960,10663121.0,NaN
436,Aruba,AW,1961,11308674.0,NaN
437,Aruba,AW,1962,12017616.0,NaN
438,Aruba,AW,1963,12696704.0,NaN
...,...,...,...,...,...
9605,Sint Maarten (Dutch part),SX,2016,64902615.0,NaN
9606,Sint Maarten (Dutch part),SX,2017,65653735.0,NaN
9607,Sint Maarten (Dutch part),SX,2018,66375543.0,NaN
9608,Sint Maarten (Dutch part),SX,2019,67080068.0,NaN


In [17]:
import datetime
df_temp_59 = df_temp
df_temp_59[datetime.df_temp['dt'].year > 1959]
print(df_temp_59)

AttributeError: ignored

In [15]:

check_nan =df_temp['AverageTemperature'].isnull()
print(check_nan)
df_temp.loc[df_temp['AverageTemperature'].isnull()]

# printing the result

0         False
1          True
2          True
3          True
4          True
          ...  
577457    False
577458    False
577459    False
577460    False
577461     True
Name: AverageTemperature, Length: 577462, dtype: bool


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland
9,1744-08-01,NaN,NaN,Åland
...,...,...,...,...
575800,1875-04-01,NaN,NaN,Zimbabwe
575852,1879-08-01,NaN,NaN,Zimbabwe
575853,1879-09-01,NaN,NaN,Zimbabwe
575854,1879-10-01,NaN,NaN,Zimbabwe
